# SQL


Книжная компания купила крупный сервис для чтения книг по подписке. Задача — проанализировать базу данных.
В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

#### Оглавление: 
<a class="anchor" id="step_0"></a>  

* [1. Подключение к базе;](#step_1)
* [2. Описание данных;](#step_2)
 * [2.1. Книги;](#step_2.1)
 * [2.2. Авторы;](#step_2.2)
 * [2.3. Издательства](#step_3.3)
 * [2.4. Рейтинги](#step_3.4)
 * [2.5. Рецензии](#step_3.5)
* [3. Анализ данных](#step_4) 
 * [3.1. Сколько книг вышло после 1 января 2000 года;](#step_3.1)
 * [3.2. Количество обзоров и средние оценки;](#step_3.2)
 * [3.3. Издательство выпустившее наибольшее число книг;](#step_3.3)
 * [3.4. Автор с самой высокой средней оценкой;](#step_3.4)
 * [3.5. Среднее количество обзоров от пользователей](#step_3.5)
* [4. Выводы и рекомендации](#step_4)

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

## 1. Подключение у базе: 
[к оглавлению](#step_0)
<a class="anchor" id="step_1"></a>  

In [2]:
# Формируем подключение
db_config = {'user': 'praktikum_student', # имя пользователя
            'pwd': 'Sdf4$2;d-d30pp', # пароль
            'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
            'port': 6432, # порт подключения
            'db': 'data-analyst-final-project-db'} # название базы данных

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                db_config['pwd'],
                                                db_config['host'],
                                                db_config['port'],
                                                db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [3]:
# Соберем функцию, для вывода данных
def get_query(query):
    query = pd.io.sql.read_sql(query, con = engine)
    return query

## 2. Описание данных: 
[к оглавлению](#step_0)
<a class="anchor" id="step_2"></a>  

Для каждого запроса сформируем переменную и передадим в функцию для вывода таблицы

### 2.1. Книги: 
[к оглавлению](#step_0)
<a class="anchor" id="step_2.1"></a>  

In [4]:
books_tab = ''' 
SELECT *
FROM 
    books
ORDER BY 
    publication_date DESC
LIMIT 5
'''
get_query(books_tab)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,43,377,A Quick Bite (Argeneau #1),360,2020-03-31,28
1,635,166,The Art of Loving,192,2019-08-06,130
2,445,612,Monster,281,2019-03-05,14
3,293,80,Ham on Rye,288,2014-07-29,99
4,993,80,Women,291,2014-07-29,99


**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

In [5]:
books_info = ''' 
SELECT
    COUNT (book_id) AS book_cnt,
    COUNT (DISTINCT title) AS title_cnt,
    COUNT (author_id) AS author_cnt,
    COUNT (publisher_id) AS publisher_cnt,
    MIN (num_pages) AS min_pages,
    MAX (num_pages) AS max_pages,
    MIN (publication_date) AS first_date,
    MAX (publication_date) AS last_date 
FROM books
'''
get_query(books_info)

,book_cnt,title_cnt,author_cnt,publisher_cnt,min_pages,max_pages,first_date,last_date
0,1000,999,1000,1000,14,2690,1952-12-01,2020-03-31


Более детальная информация о книгах 

>- Всего 1000 id книг, причем уникальных названий 999. 
>- 340 издательств
>- Самая тонкая книга в 14 страниц
>- Самая толстая в 2690 страниц
>- Даты изданий от 1952 года до 2020

В общем всего понемногу. Проверим почему число книг 1000, а наименований 999, похоже есть два дубля

In [6]:
books_duplicat = ''' 
SELECT *
FROM 
    books
WHERE
    title = (SELECT Sub.title
             FROM 
                (SELECT title, 
                 COUNT(*) AS title_cnt
                 FROM 
                     books
                 GROUP BY 
                     title
                 HAVING COUNT(*)>1) AS Sub);
'''
get_query(books_duplicat)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,426,39,Memoirs of a Geisha,434,2005-11-15,241
1,427,39,Memoirs of a Geisha,503,2005-11-22,311


Под названием "Мемуары гейши" две книги с разным числом страниц и разными датами публикации и разными издателями. Это не дубли.

### 2.2. Авторы: 
[к оглавлению](#step_0)
<a class="anchor" id="step_2.2"></a>  

In [7]:
authors_tab = ''' 
SELECT *
FROM authors
ORDER BY author
LIMIT 5
'''
get_query(authors_tab)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

In [8]:
authors_info = ''' 
SELECT
COUNT(author_id) as author_cnt,
COUNT(DISTINCT author) as author_names
FROM authors;
'''
get_query(authors_info)

,author_cnt,author_names
0,636,636


Всего представлено 636 авторов и они не дублируются.

### 2.3. Издательства: 
[к оглавлению](#step_0)
<a class="anchor" id="step_2.3"></a>  

In [9]:
publishers_tab = ''' 
SELECT *
FROM publishers
ORDER BY publisher
LIMIT 5
'''
get_query(publishers_tab)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;


In [10]:
publishers_info = ''' 
SELECT
COUNT(publisher_id) as publisher_cnt,
COUNT(DISTINCT publisher) as publisher_names
FROM publishers
'''
get_query(publishers_info)

,publisher_cnt,publisher_names
0,340,340


Всего представлено 340 издательств и они не дублируются.

### 2.4. Рейтинги: 
[к оглавлению](#step_0)
<a class="anchor" id="step_2.4"></a>  

In [11]:
ratings_tab = ''' 
SELECT *
FROM ratings
LIMIT 5
'''
get_query(ratings_tab)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

In [12]:
ratings_info = ''' 
SELECT
    COUNT (rating) AS rating_cnt,
    COUNT (DISTINCT book_id) AS books_rated,
    MIN (rating) AS min_rating,
    MAX (rating) AS max_rating
FROM ratings
'''
get_query(ratings_info)

,rating_cnt,books_rated,min_rating,max_rating
0,6456,1000,1,5


Всего в таблице 6456 оценок ко всем 1000 книгам. Максимальная __1__ минимальная __5__. Что в общем то не удивительно.

### 2.5. Рецензии: 
[к оглавлению](#step_0)
<a class="anchor" id="step_2.5"></a>  

In [13]:
reviews_tab = ''' 
SELECT *
FROM reviews
LIMIT 5
'''
get_query(reviews_tab)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

In [14]:
reviews_info = ''' 
SELECT 
COUNT (review_id) AS review_cnt
FROM reviews;
'''
get_query(reviews_info)

,review_cnt
0,2793


Всего в таблице 2793 рецензии

Данные загружены, проверены. Приступаем к анализу.

## Анализ данных: 
[к оглавлению](#step_0)
<a class="anchor" id="step_з"></a>  

Используя базу данных, получим необходимые выводы для принятия решений.

### 3.1. Сколько книг вышло после 1 января 2000 года:
[к оглавлению](#step_0)
<a class="anchor" id="step_3.1"></a>  

In [15]:
books_count = ''' 
SELECT count(b.book_id) AS "Выпущено книг"
FROM books b 
WHERE b.publication_date::date > '2000-01-01';
'''
get_query(books_count)

,Выпущено книг
0,819


Из 1000 книг, находящихся в распоряжении сервиса, после 2000 года выпущено 819

### 3.2. Количество обзоров и средние оценки:
[к оглавлению](#step_0)
<a class="anchor" id="step_3.2"></a>  

In [16]:
books_reviews = '''
SELECT
title AS "Назвние",
cnt_rev AS "Количество обзоров",
avg_rat AS "Средний рейтинг"
FROM books b 
JOIN (
    SELECT r.book_id,
            count(r.review_id) AS cnt_rev
            FROM reviews r
            GROUP BY (r.book_id)) AS t1 ON b.book_id = t1.book_id
JOIN (
    SELECT r.book_id,
            round(avg(r.rating)::NUMERIC, 2) AS avg_rat
            FROM ratings r 
            GROUP BY (r.book_id)) AS t2 ON b.book_id = t2.book_id
ORDER BY (cnt_rev, avg_rat) DESC
LIMIT 10;
'''
get_query(books_reviews)

,Назвние,Количество обзоров,Средний рейтинг
0,Twilight (Twilight #1),7,3.66
1,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.41
2,Harry Potter and the Chamber of Secrets (Harry...,6,4.29
3,The Book Thief,6,4.26
4,The Glass Castle,6,4.21
5,Outlander (Outlander #1),6,4.13
6,The Hobbit or There and Back Again,6,4.13
7,The Curious Incident of the Dog in the Night-Time,6,4.08
8,The Lightning Thief (Percy Jackson and the Oly...,6,4.08
9,Water for Elephants,6,3.98


In [17]:
books_ratings = '''
SELECT
title AS "Назвние",
cnt_rev AS "Количество обзоров",
avg_rat AS "Средний рейтинг"
FROM books b 
JOIN (
    SELECT r.book_id,
            count(r.review_id) AS cnt_rev
            FROM reviews r
            GROUP BY (r.book_id)) AS t1 ON b.book_id = t1.book_id
JOIN (
    SELECT r.book_id,
            round(avg(r.rating)::NUMERIC, 2) AS avg_rat
            FROM ratings r 
            GROUP BY (r.book_id)) AS t2 ON b.book_id = t2.book_id
ORDER BY (avg_rat, cnt_rev) DESC
LIMIT 10;
'''
get_query(books_ratings)

,Назвние,Количество обзоров,Средний рейтинг
0,A Dirty Job (Grim Reaper #1),4,5.0
1,School's Out—Forever (Maximum Ride #2),3,5.0
2,Moneyball: The Art of Winning an Unfair Game,3,5.0
3,Evening Class,2,5.0
4,March,2,5.0
5,Light in August,2,5.0
6,Wherever You Go There You Are: Mindfulness Me...,2,5.0
7,Act of Treason (Mitch Rapp #9),2,5.0
8,Dead Souls,2,5.0
9,The Demon-Haunted World: Science as a Candle i...,2,5.0


Самые обозреваемые книги "Сумерки" "Ешь молись любись" и серии про Гарри Поттера. Самые популярные - "Добро пожаловать в искушение". Примечательно что, Количество обзоров не гарантируем высокую оценку, а скорее наоборот.

### 3.3. Издательство выпустившее наибольшее число книг:
[к оглавлению](#step_0)
<a class="anchor" id="step_3.3"></a>  

In [18]:
top_publishers = '''
SELECT p.publisher,
count(b.book_id)
FROM books b 
JOIN publishers p ON b.publisher_id = p.publisher_id 
WHERE b.num_pages > 50
GROUP BY publisher
ORDER BY "count" DESC 
LIMIT 3
'''
get_query(top_publishers)

,publisher,count
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25


Лидер по выпуску книг издательство "Pinguin Books" - выпуск 42 книг

### 3.4. Автор с самой высокой средней оценкой:
[к оглавлению](#step_0)
<a class="anchor" id="step_3.4"></a>  

In [19]:
top_author = '''
SELECT a.author,
avg(avg_rat) AS avg_rat
FROM books b 
JOIN (
    SELECT book_id,
    avg(rating) AS avg_rat
    FROM ratings r 
    GROUP BY book_id
    HAVING count(rating_id) > 50) AS t 
ON b.book_id = t.book_id
JOIN authors a ON b.author_id = a.author_id 
GROUP BY a.author 
ORDER BY avg(avg_rat) DESC 
'''
get_query(top_author)

,author,avg_rat
0,J.K. Rowling/Mary GrandPré,4.283844
1,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,J.R.R. Tolkien,4.258446
3,Louisa May Alcott,4.192308
4,Rick Riordan,4.080645
5,William Golding,3.901408
6,J.D. Salinger,3.825581
7,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.789474
8,William Shakespeare/Paul Werstine/Barbara A. M...,3.787879
9,Dan Brown,3.754540


Самая высокая оценка у автора Гарри Поттера - Дж. Роулинг. Почти __4.3__ балла

### 3.5. Среднее количество обзоров от пользователей:
[к оглавлению](#step_0)
<a class="anchor" id="step_3.5"></a>  

In [20]:
customers_reviews = '''
SELECT avg(cnt) FROM (
SELECT
count(review_id) AS cnt
FROM reviews r 
WHERE username IN (
    SELECT r.username
    FROM ratings r
    GROUP BY r.username
    HAVING count(r.rating_id) > 50)
GROUP BY username) AS t
'''
get_query(customers_reviews)

,avg
0,24.333333


Среднее количество обзоров оказалось 24.3 

## 4. Выводы и рекомендации: 
[к оглавлению](#step_0)
<a class="anchor" id="step_4"></a>  

__Выводы__: 
____
>- Сервис располагает преимущественно свежим фондом. Из 1000 книг 819 выпущены после 2000 года.
>- Всего представлены 636 авторов и 340 изданий.
>- Есть тонкие книги - 14 страниц, если и фолианты
>- Все книги имеют обзоры
>- Книги с большим количеством обзоров не имеют высоких оценок
>- Клиенты в среднем обсуждают книги 24 раза

__Рекомендации__ : 
____
>- Основное направление это обчные известные авторы, популярные в последние десятилетия
>- Рекомендовать авторов из ТОП 10 во главе с мамкой Гарри Поттера
>- Не особо доверять 5 бальной оценке, лучше подождать когда книга наберет 5-6 рецензий, и тогда смотреть на балл
>- Присматриваться к обзорам активных пользователей - совершивших свыше 25. Возможно они формируют мнение
>- Так же основываясь на мнении этих пользователей составить рекомендательную модель. Какую-нибудь коллаборативную фильтрацию например